In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv',encoding='latin1')


In [3]:
df.rename(columns = {'emotion_in_tweet_is_directed_at': 'Product', 'is_there_an_emotion_directed_at_a_brand_or_product': 'Emotion', 'tweet_text': 'text'}, inplace= True)

In [4]:
df.dropna(subset=['text'], inplace = True)

In [5]:
df['text'] = df['text'].str.replace(r'[^\w\s]', '')
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.split(' ')
df.fillna('na',inplace=True)
df['test'] = ''

<ipython-input-5-e754b91c759f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'[^\w\s]', '')


In [6]:
apple = ['iphone','ipad','apple','apples','ipads']
android = ['android','google','androids','googles']
for ind , val in enumerate(df['text']):
    if df['Product'].values[ind] == 'na':
        for y in val:
            if y in apple:
                df['test'].values[ind] = y
            elif y in android:
                df['test'].values[ind] = y
                
    else:
        df['test'].values[ind] = df['Product'].values[ind]

In [7]:
df['test'] = df['test'].map(lambda x : x.lower())
df['test'].unique()

array(['iphone', 'ipad or iphone app', 'ipad', 'google', 'android',
       'apple', 'android app', 'other google product or service', '',
       'googles', 'ipads', 'apples', 'other apple product or service',
       'androids'], dtype=object)

In [8]:
df['test'] = df['test'].replace({'googles': 'google', 'apples': 'apple', 'androids': 'android','ipads':'ipad'})

In [9]:
def company_search(test):
    if test == 'iphone':
        return 'apple'
    elif test == 'apple':
        return 'apple'
    elif test == 'ipad':
        return 'apple'
    elif test == 'ipad or iphone app':
        return 'apple'
    elif test == 'itunes':
        return 'apple'
    elif test == 'other apple product or service':
        return 'apple'
    elif test == 'ios':
        return 'apple'
    elif test == 'ipadquot':
        return 'apple'
    elif test == 'applequot':
        return 'apple'
    elif test == 'iphonequot':
        return 'apple'
    elif test == 'quotiphone':
        return 'apple'
    elif test == 'iosquot':
        return 'apple'
    elif test =='quotipad':
        return 'apple'
    elif test == 'quotapple':
        return 'apple'
    elif test =='google':
        return 'google'
    elif test == 'android':
        return 'google'
    elif test == 'other google product or service':
        return 'google'
    elif test == 'android app':
        return 'google'
    elif test == 'quotgoogle':
        return 'google'
    elif test == 'googlequot':
        return 'google'
    elif test == 'androidquot':
        return 'google'
    else:
        return 'unknown'
df['company'] = df['test'].map(company_search)

In [10]:
df.Emotion.value_counts()

No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: Emotion, dtype: int64

In [11]:
df['Emotion'] = df['Emotion'].replace({'No emotion toward brand or product': 'neutral', 'I can\'t tell': 'neutral'})

In [12]:
df.Emotion.value_counts()

neutral             5544
Positive emotion    2978
Negative emotion     570
Name: Emotion, dtype: int64

In [13]:
l_encode = LabelEncoder()

In [14]:
df.Emotion = l_encode.fit_transform(df.Emotion)

In [15]:
df['target'] = df['Emotion']
df['product'] = df['test']

In [16]:
df = df.drop(['Product', 'Emotion', 'test'], axis = 1)

In [17]:
df = df[df['product'] != '']

In [18]:
df['text'] = df['text'].str.join(' ')

In [19]:
df.head()

,text,company,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,apple,0,iphone
1,jessedee know about fludapp awesome ipadiphon...,apple,1,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,apple,1,ipad
3,sxsw i hope this years festival isnt as crashy...,apple,0,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,google,1,google


In [20]:
from pickle import dump

In [21]:
dump(df, open('df_clean.pkl', 'wb'))